# class화 한 대묵 솔루션 대입

In [3]:
import pandas as pd
import time,datetime
import numpy as np
import glob
from pandas import *
import scipy.optimize
%pylab
%matplotlib inline

#timeseries data
all_data=pd.read_csv('Timeseries_data_SP500.csv')
date_list=list(set(list(all_data['DATE'])))
for i in range(len(date_list)):
    ak=date_list[i]
    date_list[i]=ak[6:]+str('-')+ak[:2]+'-'+ak[3:5]
date_list.sort()
all_data['DATE']=pd.to_datetime(all_data['DATE'])
all_data=all_data.rename(columns={'NAME':'name','DATE':'date','SEDOL':'sedol','SECTOR':'sector','BETA':'beta','ALPHA_SCORE':'as','BENCH_WEIGHT':'bw',"MCAP_Q":'mq'})

dic_data = {k: v for k, v in all_data.groupby('date')}

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


C:\ProgramData\Anaconda3\envs\py34\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['datetime', 'random', 'types', 'unique', 'info', 'test']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [4]:
#risk:cov_mat
risk_data=pd.read_csv('cov_mat_2007-01-03.csv')
risk_sedol=risk_data['ROW_INDEX'].unique().tolist()
risk_mat = np.zeros((len(risk_sedol),len(risk_sedol)))
risk_mat[np.triu_indices(len(risk_sedol), 0)] = list(risk_data['VALUE'])
irows,icols = np.triu_indices(len(risk_sedol),0)
risk_mat[icols,irows]=risk_mat[irows,icols]

In [7]:
import class_informs as inform
import pickle
import random
import heapq


sam = 200
#샘플링 수 (보통 lambda_의 0.2~0.25정도)
lambda_ = 1000
#모집단 수 (이 문제에서는 생성한 포트폴리오의 수)
ngen = 50
#반복 횟수
dim = len(dic_data[Timestamp('2007-01-03 00:00:00')])
#종목의 총 갯수
gen = 0

ex = inform.informs(10000)
f8 = open('pic/risk_mat.txt', 'rb')
risk_mat = pickle.load(f8)
ex.set_omega(risk_mat)

prob = [0]*dim
population = []
r_a = []
selected = []

def generate():
    
#   처음엔 랜덤으로 선택할 자산을 car개 선택
#   두번째부터는 확률 반영해서 선택
#   선택한게 car개 보다 적으면 확률이 큰거 추가해서 car개 맞춤
    population = []
    r_a = []
#     r_a_ranks = []
    
    for i in range(lambda_) :
        candi = []
        candi_ = []
        car = int(random.random()*20+50)
        if gen == 0 :
            candi = random.sample(range(dim), car)
            
        else :
            for j in range(dim) :
                if random.random() <= prob[j] :
                    candi.append(j)
            if len(candi)>car :
                candi = random.sample(candi, car)
            elif lan(candi) <car :
                large = heapq.nlargest(car, prob)
                candi_ = [prob.index(k) for k in large]
                candi += random.sample(set(candi_)-set(candi), car-len(candi))
        ex.set_con(cons=candi)
        ex.solve(0.01)
        
#   각 포트폴리오의 risk, as의 합 계산
    for i in range(lambda_) :
        s_rr = np.dot(population[i], risk_mat)
        s_r = np.dot(s_rr, population[i])
        s_a = np.dot(population[i], as_)
        r_a.append(s_r-s_a)
# #   각 포트폴리오의 risk-revenue 순위
#     array = numpy.array(r_a)
#     temp = array.argsort()
#     r_a_ranks = temp.argsort()
    
    return population


def update(population) :
    selected = []
    small = heapq.nsmallest(sam, r_a)
    print(min(r_a))
    
    selected = [population[r_a.index(i)] for i in small]
    
#     for i in small : 
#         selected.append(population[r_a.index(i)])
        
#     for i in range(lambda_) :
#         if r_a_ranks[i] < mu :
#             selected.append(population[i])

    for i in range(selected) :
        for j in range(dim) :
            if selected[i, j] > 0 :
                prob[j] += 1
    
    gen += 1

prob
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
population
0
r_a
0
selected
0


C:\ProgramData\Anaconda3\envs\py34\lib\site-packages\numpy\core\numeric.py:301: FutureWarning: in the future, full((492, 492), 10000) will return an array of dtype('int32')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)


In [6]:
for i in range(ngen) :
    population = generate()
    update(population)

Tried aggregator 1 time.
QCP Presolve eliminated 2119 rows and 566 columns.
Aggregator did 1 substitutions.
Reduced QCP has 936 rows, 915 columns, and 25919 nonzeros.
Reduced QCP has 2 quadratic constraints.
Presolve time = 0.13 sec. (256.52 ticks)
Parallel mode: using up to 4 threads for barrier.

***NOTE: Found 5 dense columns.

Number of nonzeros in lower triangle of A*A' = 361220
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.11 sec. (87.34 ticks)
Summary statistics for Cholesky factor:
  Threads                   = 4
  Rows in Factor            = 936
  Integer space required    = 9844
  Total non-zeros in factor = 369117
  Total FP ops to factor    = 192931447
 Itn      Primal Obj        Dual Obj  Prim Inf Upper Inf  Dual Inf Inf Ratio
   0  1.1914935e+000 -9.7842360e-001 1.22e+004 0.00e+000 1.57e+002 1.00e+000
   1  2.0190648e+000  1.3905640e+000 1.22e+004 0.00e+000 1.57e+002 1.23e+000
   2  2.7209547e+000  2.1235128e+000 5.54e+003 0.00e+000 7.11e

 Itn      Primal Obj        Dual Obj  Prim Inf Upper Inf  Dual Inf Inf Ratio
   0  8.1248111e-001 -9.7880602e-001 1.20e+004 0.00e+000 1.50e+002 1.00e+000
   1  1.8649136e+000  1.4360011e+000 1.20e+004 0.00e+000 1.50e+002 1.21e+000
   2  2.5349686e+000  2.1954547e+000 5.39e+003 0.00e+000 6.75e+001 7.84e-001
   3  3.9740160e+000  3.6609593e+000 6.93e+003 0.00e+000 8.68e+001 1.34e+000
   4  5.4486707e+000  5.0701220e+000 4.56e+003 0.00e+000 5.71e+001 4.42e+000
   5  7.1211389e+000  6.9875126e+000 2.60e+003 0.00e+000 3.25e+001 3.10e+001
   6  8.3242272e+000  8.3040876e+000 7.12e+002 0.00e+000 8.91e+000 2.32e+002
   7  8.4772805e+000  8.4720435e+000 1.05e+002 0.00e+000 1.31e+000 6.96e+002
   8  8.5120451e+000  8.5099466e+000 2.87e+001 0.00e+000 3.59e-001 1.59e+003
   9  8.5287309e+000  8.5282198e+000 1.17e+001 0.00e+000 1.47e-001 6.32e+003
  10  8.5344972e+000  8.5344707e+000 2.87e+000 0.00e+000 3.60e-002 1.40e+005
  11  8.5348119e+000  8.5348106e+000 1.44e-001 0.00e+000 1.80e-003 2.90e+006

KeyboardInterrupt: 